# HF STO-3G calculation for HeH+



In [16]:
import numpy as np
import scipy.special as sp
import sympy as sy
import matplotlib.pyplot as plt

Here is the Fortran code written out http://www.ccl.net/cca/software/SOURCES/FORTRAN/szabo/szabo.f with the output http://www.ccl.net/cca/software/SOURCES/FORTRAN/szabo/out.txt 

Need to go through the book and fill this in properly. Also need to put in graphs.

## Integrals

We make use of Gaussian orbitals fitted to Slater type orbitals of the form

$$g_{1s}(\alpha)=(2\alpha/\pi)^{3/4}exp(-\alpha r^{2}) $$

Where the exponents are 

In [62]:
def S_int(A,B,Rab2):
    # Calculates the overlaps for un-normalized primitives
    return (np.pi/(A+B))**1.5*np.exp(-A*B*Rab2/(A+B))

In [63]:
def T_int(A,B,Rab2):
    # Calculates the kinetic energy integrals for un-normalised primitives
    return A*B/(A+B)*(3.0-2.0*A*B*Rab2/(A+B))*(np.pi/(A+B))**1.5*np.exp(-A*B*Rab2/(A+B))

In [197]:
def V_int(A,B,Rab2,Rcp2,Zc):
    # Calculates the un-normalised nuclear attraction integrals
    V = 2.0*np.pi/(A+B)*F0((A+B)*Rcp2)*np.exp(-A*B*Rab2/(A+B))
    return -V*Zc

def F0(t):
    # F function for 1s orbital
    if (t<1e-6):
        return 1.0-t/3.0
    else:
        return 0.5*(np.pi/t)**0.5*sp.erf(t**0.5)
def erf(t):
    P = 0.3275911
    A = [0.254829592,-0.284496736,1.421413741,-1.453152027,1.061405429]
    T = 1.0/(1+P*t)
    TN=T
    POLY = A[0]*TN
    for i in range(1,5):
        TN=TN*T
        POLY=POLY*A[i]*TN
    return 1.0-POLY*np.exp(-t*t)
    
    

In [182]:
F0(50)

0.12533141373155002

In [ ]:

      IMPLICIT DOUBLE PRECISION(A-H,O-Z)
      DIMENSION A(5)
      DATA P/0.3275911D0/
      DATA A/0.254829592D0,-0.284496736D0,1.421413741D0,
     $ -1.453152027D0,1.061405429D0/
      T=1.0D0/(1.0D0+P*ARG)
      TN=T
      POLY=A(1)*TN
      DO 10 I=2,5
      TN=TN*T
      POLY=POLY+A(I)*TN
   10 CONTINUE
      DERFOTHER=1.0D0-POLY*DEXP(-ARG*ARG)
      RETURN
      END

In [202]:
def TwoE(A,B,C,D,Rab2,Rcd2,Rpq2):
    # Calculate two electron integrals
    # A,B,C,D are the exponents alpha, beta, etc.
    # Rab2 equals squared distance between centre A and centre B
    return 2.0*(np.pi**2.5)/((A+B)*(C+D)*np.sqrt(A+B+C+D))*F0((A+B)*(C+D)*Rpq2/(A+B+C+D))*np.exp(-A*B*Rab2/(A+B)-C*D*Rcd2/(C+D))

In [ ]:
C*********************************************************************
      FUNCTION TWOE(A,B,C,D,RAB2,RCD2,RPQ2)
C
C CALCULATES TWO-ELECTRON INTEGRALS FOR UN-NORMALIZED PRIMITIVES
C A,B,C,D ARE THE EXPONENTS ALPHA, BETA, ETC.
C RAB2 EQUALS SQUARED DISTANCE BETWEEN CENTER A AND CENTER B, ETC.
C*********************************************************************

      IMPLICIT DOUBLE PRECISION(A-H,O-Z)
      DATA PI/3.1415926535898D0/
      TWOE=2.0D0*(PI**2.5D0)/((A+B)*(C+D)*DSQRT(A+B+C+D))
     $ *F0((A+B)*(C+D)*RPQ2/(A+B+C+D))
     $ *DEXP(-A*B*RAB2/(A+B)-C*D*RCD2/(C+D))
      RETURN
      END

In [235]:
def Intgrl(IOP,N,R,Zeta1,Zeta2,Za,Zb):
    
    global S12,T11,T12,T22,V11A,V12A,V22A,V11B,V12B,V22B,V1111,V2111,V2121,V2211,V2221,V2222
    
    
    S12 = 0.0
    T11 = 0.0
    T12 = 0.0
    T12 = 0.0
    T22 = 0.0
    V11A = 0.0
    V12A = 0.0
    V22A = 0.0
    V11B = 0.0
    V12B = 0.0
    V22B = 0.0
    V1111 = 0.0
    V2111 = 0.0
    V2121 = 0.0
    V2211 = 0.0
    V2221 = 0.0
    V2222 = 0.0

    
    R2 = R*R
    
    Coeff = np.array([[1.00000,0.0000000,0.000000],
                      [0.678914,0.430129,0.000000],
                      [0.444635,0.535328,0.154329]])
    
    Expon = np.array([[0.270950,0.000000,0.000000],
                      [0.151623,0.851819,0.000000],
                      [0.109818,0.405771,2.227660]])
    D1 = np.zeros([3])
    A1 = np.zeros([3])
    D2 = np.zeros([3])
    A2 = np.zeros([3])
    
    for i in range(N):
        A1[i] = Expon[N-1,i]*(Zeta1**2)
        D1[i] = Coeff[N-1,i]*((2.0*A1[i]/np.pi)**0.75)
        A2[i] = Expon[N-1,i]*(Zeta2**2)
        D2[i] = Coeff[N-1,i]*((2.0*A2[i]/np.pi)**0.75)
    
    # Calculate one electron integrals 
    # Centre A is first atom centre B is second atom
    # Origin is on second atom
    # V12A - off diagonal nuclear attraction to centre A etc.
    for i in range(N):
        for j in range(N):
            # Rap2 - squared distance between centre A and centre P
            Rap = A2[j]*R/(A1[i]+A2[j])
            Rap2 = Rap**2
            Rbp2 = (R-Rap)**2
            S12 = S12 + S_int(A1[i],A2[j],R2)*D1[i]*D2[j]
            T11 = T11 + T_int(A1[i],A1[j],0.0)*D1[i]*D1[j]
            T12 = T12 + T_int(A1[i],A2[j],R2)*D1[i]*D2[j]
            T22 = T22 + T_int(A2[i],A2[j],0.0)*D2[i]*D2[j]
            V11A = V11A + V_int(A1[i],A1[j],0.0,0.0,Za)*D1[i]*D1[j]
            V12A = V12A + V_int(A1[i],A2[j],R2,Rap2,Za)*D1[i]*D2[j]
            V22A = V22A + V_int(A2[i],A2[j],0.0,R2,Za)*D2[i]*D2[j]
            V11B = V11B + V_int(A1[i],A1[j],0.0,R2,Zb)*D1[i]*D1[j]
            V12B = V12B + V_int(A1[i],A2[j],R2,Rbp2,Zb)*D1[i]*D2[j]
            V22B = V22B + V_int(A2[i],A2[j],0.0,0.0,Zb)*D2[i]*D2[j]
    
    # Calculate two electron integrals
    
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for l in range(N):
                    Rap = A2[i]*R/(A2[i]+A1[j])
                    Rbp = R - Rap
                    Raq = A2[k]*R/(A2[k]+A1[l])
                    Rbq = R - Raq
                    Rpq = Rap - Raq
                    Rap2 = Rap*Rap
                    Rbp2 = Rbp*Rbp
                    Raq2 = Raq*Raq
                    Rbq2 = Rbq*Rbq
                    Rpq2 = Rpq*Rpq
                    # Check this as Rbp2 is not used but calculated
                    V1111 = V1111 + TwoE(A1[i],A1[j],A1[k],A1[l],0.0,0.0,0.0)*D1[i]*D1[j]*D1[k]*D1[l]
                    V2111 = V2111 + TwoE(A2[i],A1[j],A1[k],A1[l],R2,0.0,Rap2)*D2[i]*D1[j]*D1[k]*D1[l]
                    V2121 = V2121 + TwoE(A2[i],A1[j],A2[k],A1[l],R2,R2,Rpq2)*D2[i]*D1[j]*D2[k]*D1[l]
                    V2211 = V2211 + TwoE(A2[i],A2[j],A1[k],A1[l],0.0,0.0,R2)*D2[i]*D2[j]*D1[k]*D1[l]
                    V2221 = V2221 + TwoE(A2[i],A2[j],A2[k],A1[l],0.0,R2,Rbq2)*D2[i]*D2[j]*D2[k]*D1[l]
                    V2222 = V2222 + TwoE(A2[i],A2[j],A2[k],A2[l],0.0,0.0,0.0)*D2[i]*D2[j]*D2[k]*D2[l]
    return 

In [227]:
X.T

array([[ 0.58706428,  0.58706428],
       [ 0.95413107, -0.95413107]])

In [445]:
def Colect(IOP,N,R,Zeta1,Zeta2,Za,Zb):
    # Takes the basic integrals and assembles the relevant matrices, that are S,H,X,XT and Two electron integrals
    
    # Form core hamiltonian
    H[0,0] = T11+V11A+V11B
    H[0,1] = T12+V12A+V12B
    H[1,0] = H[0,1]
    H[1,1] = T22+V22A+V22B

    # Form overlap matrix
    S[0,0] = 1.0
    S[0,1] = S12
    S[1,0] = S12
    S[1,1] = 1.0
    
    X[0,0] = 1.0/np.sqrt(2.0*(1.0+S12))
    X[1,0] = X[0,0]
    X[0,1] = 1.0/np.sqrt(2.0*(1.0-S12))
    X[1,1] = -X[0,1]
    
    
    
    TT[0,0,0,0] = V1111
    TT[1,0,0,0] = V2111
    TT[0,1,0,0] = V2111
    TT[0,0,1,0] = V2111
    TT[0,0,0,1] = V2111
    TT[1,0,1,0] = V2121
    TT[0,1,1,0] = V2121
    TT[1,0,0,1] = V2121
    TT[0,1,0,1] = V2121
    TT[1,1,0,0] = V2211
    TT[0,0,1,1] = V2211
    TT[1,1,1,0] = V2221
    TT[1,1,0,1] = V2221
    TT[1,0,1,1] = V2221
    TT[0,1,1,1] = V2221
    TT[1,1,1,1] = V2222
    
    
    

In [446]:

TT

array([[[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 1.30715161,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]]])

In [490]:
def SCF(IOP,N,R,Zeta1,Zeta2,Za,Zb,G):
    # Performs the SCF iterations
    Crit = 1e-4 # Convergence critera
    Maxit = 25 # Maximum number of iterations
    Iter=0
    # Use core hamiltonian for initial guess of F, I.E. (P=0)
    P = np.zeros([2,2])
    Energy = 0.0  
    while (Iter<Maxit):
        Iter += 1
        print Iter
        #Form two electron part of Fock matrix from P
        G = np.zeros([2,2])
        for i in range(2):
            for j in range(2):
                for k in range(2):
                    for l in range(2):
                        G[i,j]=G[i,j]+P[k,l]*(TT[i,j,k,l]-0.5*TT[i,j,k,l])
        print "P",P
        print 'G',G
        # Add core hamiltonian to get fock matrix
        for i in range(2):
            for j in range(2):
                F[i,j]=H[i,j]+G[i,j]
        
        print 'F',F
        
        # Calculate electronic energy
        #for i in range(2):
        #    for j in range(2):
        #        Energy += 0.5*P[i,j]*(H[i,j]+F[i,j])
        Energy = np.sum(0.5*P*(H+F))        
        
        print 'Electronic energy = ',Energy
        
        # Transform Fock matrix using G for temporary storage
        #Mult(F,X,G,2,2)
        #Mult(X.T,G,Fprime,2,2)
        
        G = np.matmul(F,X)
        Fprime = np.matmul(X.T,G)
        
        
        
        #Diagonalise transformed Fock matrix
        Diag(Fprime,Cprime,E)
        
        
        #Transform eigen vectors to get matrix C
        C = np.matmul(X,Cprime)
        
        Oldp = np.array(P)
        print "Oldp",Oldp
        P= np.zeros([2,2])
        
        #Form new density matrix
        for i in range(2):
            for j in range(2):
                #Save present density matrix before creating a new one
                for k in range(1):
                    P[i,j] += 2.0*C[i,k]*C[j,k]
                    
        print "P",P
        Delta = 0.0
        
        #Calculate delta
        #for i in range(2):
        #    for j in range(2):
        #        Delta += (P[i,j]-Oldp[i,j])**2
        
        Delta = (P-Oldp)
        
        Delta = np.sqrt(np.sum(Delta**2)/4.0)
        print "Delta",Delta
        print "Require converege = 1E-4"
        
        
        #Check for convergence
        if (Delta<Crit):
            # Add nuclear repulsion to get total energy
            Energytot = Energy+Za*Zb/R
            print "Calculation converged with electronic energy:",Energy
            print "Calculation converged with total energy:",Energytot
            
            print "Mulliken populations"
            print np.matmul()
            
            break

In [491]:
P = np.array([[ 1.72662709,0.25985101],[ 0.25985100,0.03910662]])

G[0,0]=G[0,0]+P[1,0]*(TT[1,0,1,0]-0.5*TT[1,0,1,0])

In [492]:
G

array([[ 0.02303152,  0.        ],
       [ 0.        ,  0.        ]])

In [493]:
def FormG():
    # Calculate the G matrix from the density matrix and two electron integals
    
    
    for i in range(2):
        for j in range(2):
            G[i,j]=0.0
            for k in range(2):
                for l in range(2):
                    G[i,j]=G[i,j]+P[k,l]*(TT[i,j,k,l]-0.5*TT[i,j,k,l])
                    
def Mult(A,B,C_,IM,M):
    #Multiples two square matrices A and B to get C
    for i in range(M):
        for j in range(M):
            for k in range(M):
                C_[i,j] = A[i,j]*B[i,j]
                
def Diag(Fprime,Cprime,E):
    # Diagonalises F to give eigenvectors in C and eigen values in E, theta is the angle describing the solution
    import math
    # Angle for heteronuclear diatonic
    Theta = 0.5*math.atan(2.0*Fprime[0,1]/(Fprime[0,0]-Fprime[1,1]))
    print 'Theta', Theta
    
    Cprime[0,0] = np.cos(Theta)
    Cprime[1,0] = np.sin(Theta)
    Cprime[0,1] = np.sin(Theta)
    Cprime[1,1] = -np.cos(Theta)
    
    E[0,0] = Fprime[0,0]*np.cos(Theta)**2+Fprime[1,1]*np.sin(Theta)**2+Fprime[0,1]*np.sin(2.0*Theta)
    E[1,1] = Fprime[1,1]*np.cos(Theta)**2+Fprime[0,0]*np.sin(Theta)**2-Fprime[0,1]*np.sin(2.0*Theta)
    
    if (E[1,1] <= E[0,0]):
        Temp = E[1,1]
        E[1,1] = E[0,0]
        E[0,0] = Temp
        Temp = Cprime[0,1]
        Cprime[0,1] = Cprime[0,0]
        Cprime[0,0] = Temp
        Temp = Cprime[1,1]
        Cprime[1,1]=Cprime[1,0]
        Cprime[1,0]=Temp
    return

In [494]:
def HFCALC(IOP,N,R,Zeta1,Zeta2,Za,Zb,G):
    # Calculate one and two electron integrals
    Intgrl(IOP,N,R,Zeta1,Zeta2,Za,Zb)
    # Put all integals into array
    Colect(IOP,N,R,Zeta1,Zeta2,Za,Zb)
    # Perform the SCF calculation
    SCF(IOP,N,R,Zeta1,Zeta2,Za,Zb,G)
    return

In [495]:
global H,S,X,XT,TT,G,C,P,Oldp,F,Fprime,Cprime,E

H = np.zeros([2,2])
S = np.zeros([2,2])
X = np.zeros([2,2])
XT = np.zeros([2,2])
TT = np.zeros([2,2,2,2])
G = np.zeros([2,2])
C = np.zeros([2,2])

P = np.zeros([2,2])
Oldp = np.zeros([2,2])
F = np.zeros([2,2])
Fprime = np.zeros([2,2])
Cprime = np.zeros([2,2])
E = np.zeros([2,2])

Energy = 0.0
Delta = 0.0

IOP = 2
N = 3
R = 1.4632
Zeta1 = 2.0925
Zeta2 = 1.24
Za = 2.0
Zb = 1.0
HFCALC(IOP,N,R,Zeta1,Zeta2,Za,Zb,G)



1
P [[ 0.  0.]
 [ 0.  0.]]
G [[ 0.  0.]
 [ 0.  0.]]
F [[-2.65274472 -1.34720517]
 [-1.34720517 -1.7318284 ]]
Electronic energy =  0.0
Theta 0.426436985386
Oldp [[ 0.  0.]
 [ 0.  0.]]
P [[ 1.72662709  0.25985101]
 [ 0.25985101  0.03910662]]
Delta 0.882866853014
Require converege = 1E-4
2
P [[ 1.72662709  0.25985101]
 [ 0.25985101  0.03910662]]
G [[ 1.25395267  0.42966882]
 [ 0.42966882  0.61907821]]
F [[-1.39879206 -0.91753635]
 [-0.91753635 -1.11275019]]
Electronic energy =  -4.14186287613
Theta 0.174207363201
Oldp [[ 1.72662709  0.25985101]
 [ 0.25985101  0.03910662]]
P [[ 1.10575051  0.61388178]
 [ 0.61388178  0.34081001]]
Delta 0.42637666285
Require converege = 1E-4
3
P [[ 1.10575051  0.61388178]
 [ 0.61388178  0.34081001]]
G [[ 1.09434447  0.40371333]
 [ 0.40371333  0.65828055]]
F [[-1.55840025 -0.94349184]
 [-0.94349184 -1.07354785]]
Electronic energy =  -4.21250515815
Theta 0.276692536686
Oldp [[ 1.10575051  0.61388178]
 [ 0.61388178  0.34081001]]
P [[ 1.36250446  0.501979  ]
 [ 